In [1]:

%pprint
import sys
if (osp.join(os.pardir, 'py') not in sys.path): sys.path.insert(1, osp.join(os.pardir, 'py'))

Pretty printing has been turned OFF


In [2]:

import os.path as osp
shared_folder = osp.abspath(osp.join(os.pardir, 'share')); assert osp.exists(shared_folder)
import sys
if shared_folder not in sys.path: sys.path.insert(1, shared_folder)
from notebook_utils import NotebookUtilities
nu = NotebookUtilities(
    data_folder_path=osp.abspath(osp.join(os.pardir, 'data')),
    saves_folder_path=osp.abspath(osp.join(os.pardir, 'saves'))
)

# Import needed libraries
import os, re
import pandas as pd

In [3]:

# https://github.com/thelabcat/word-search-generator/tree/main

# Import modules
import random
from tkinter import *
from tkinter import messagebox as mb

# Coordinates start at top left, and are positive right down

DIRECTIONS=[(1, 0), (1, 1), (0, 1), (-1, 1), (-1, 0), (-1, -1), (0, -1), (1, -1)] # Directions in the form of (dx, dy)
EASY_DIRECTIONS=DIRECTIONS[:3] # Easy mode directions
SIZE_FAC_OPTIONS=tuple(range(2, 10)) # Ratio of total characters in puzzle to characters that are actually words
SIZE_FAC_DEFAULT=4
ALL_CHARS="ABCDEFGHIJKLMNOPQRSTUVWXYZ" # Filling characters and limiter for input words (modify this to allow for punctuation

class Generator(object):
    def get_puzzle_dim(self, words, size_fac=SIZE_FAC_DEFAULT):
        """Calculate puzzle dimension by word list (returns one side of a square)"""
        # Count all the letters
        total = 0
        for word in words:
            for letter in word:
                total += 1

        # Return either the side of a square large enough for all the characters times size_fac, or the length of the longest word
        return max((int((total * size_fac) ** 0.5), len(max(words, key=len))))

    def create_empty_table(self, dim):
        # Create the empty 2D table to build the puzzle. referenced by table[y][x]=" "
        table = []
        for y in range(dim):
            table.append([])
            for x in range(dim): table[-1].append(" ")
        
        return table
    
    def copy_table(self, table):
        """Independently duplicate table data"""
        return eval(str(table)) # um, hehe?
        
        # Jumped code in case previous is buggy
        copy=[]
        for row in table:
            copy.append([])
            for column in row: copy[-1].append(column[:]) # Duplicate each character independently
        
        return copy   

    def create_blank_tried_pos(self, words):
        """Create a new tried positions dictionary"""
        dictionary={}
        for word in words: dictionary[word]=[]
        
        return dictionary

    def all_random_coords(self, dim):
        xlist=list(range(dim))
        random.shuffle(xlist)
        ylist=list(range(dim))
        random.shuffle(ylist)
        
        return xlist, ylist

    def gen_word_search(self, words, directions=None, size_fac=2):
        """Generate a word search puzzle. Returns a 2D list table
    with empty spaces to be filled with random characters.
    Directions defaults to DIRECTIONS[:]"""

        # Default to all directions
        if not directions: directions = DIRECTIONS[:]

        dim=self.get_puzzle_dim(words, size_fac) # Generate puzzle dimension
        table=self.create_empty_table(dim)
        table_history={}

        # Dict of positions for each word that have been tried but hurt future words. Future word values should ALWAYS be empty.
        tried_positions=self.create_blank_tried_pos(words)

        current_index=0
        while current_index<len(words): # place every word

            word=words[current_index]

            table_history[word]=self.copy_table(table) # Record previous table state

            # Randomly shuffle all coordinate possibilities
            xlist, ylist=self.all_random_coords(dim)

            # For all possible positions (randomized)...
            for x in xlist:
                for y in ylist:

                    # For all possible directions (randomized)...
                    random.shuffle(directions)
                    
                    success = False # If all directions for this positionare used, this will be the default value
                    
                    for direction in directions:
                        
                        if (x, y, direction) in tried_positions[word]: continue # We already tried this direction
                        
                        tcopy = self.copy_table(table) # Copy the table
                        pos = [x, y] # Current position in the word. To be incremented forward by direction for each letter
                        success = True # Assume that the letter placing loop will succeed ;-D
                        
                        for letter in word: # Move forward in the word
                            try: # check that the index is valid. REMEMBER: table must be indexed [y][x]
                                if pos[0] < 0 or pos[1] < 0: raise IndexError
                                tcopy[pos[1]][pos[0]]
                            except IndexError: # Position does not exist
                                success = False
                                break

                            # Try placing the letter...
                            if tcopy[pos[1]][pos[0]] == " ": tcopy[pos[1]][pos[0]] = letter
                                
                            # Position was assigned, but is already this letter. Okay :-)
                            elif tcopy[pos[1]][pos[0]] == letter: continue
                            
                            else: # Position is occupied by a different letter. Try again...
                                success = False
                                break

                            # Step the position forward by direction
                            pos[0] += direction[0]
                            pos[1] += direction[1]
                            
                        # Word successfully placed at this orientation
                        if success: break
                    
                    # ...and at this Y position
                    if success: break
                
                # ...and at this X position
                if success: break
                
            if success: # ...So, write changes to the table
                table=self.copy_table(tcopy)
                tried_positions[word].append((x, y, direction)) # Record the tried position for potential FUTURE use
                current_index+=1
                
            else: # No position or orientation worked, so backtrack
                current_index-=1
                tried_positions[word]=[] # If we're backing up in the word list, any previously tried placing positions for this word are now invalid
                table=self.copy_table(table_history[word])
                
                if current_index<0: # Nothing worked all the way back through the first word, the puzzle is too small
                    dim+=1
                    table=self.create_empty_table(dim)
                    table_history={} # May be no reason to clear this
                    current_index=0
            
        return table # All finished :-D

In [4]:

g = Generator()
[f'g.{fn}' for fn in dir(g) if not fn.startswith('_')]

['g.all_random_coords', 'g.copy_table', 'g.create_blank_tried_pos', 'g.create_empty_table', 'g.gen_word_search', 'g.get_puzzle_dim']

In [5]:

from openpyxl import Workbook

# g.gen_word_search?

In [7]:

background_words_list = [
    'accurate', 'address', 'afford', 'alert', 'analyze', 'ancestor', 'annual', 'apparent', 'appropriate', 'arena', 'arrest', 'ascend', 'assist', 'attempt', 'attentive', 'attractive', 'awkward', 'baggage',
    'basic', 'benefit', 'blend', 'blossom', 'burrow', 'calculate', 'capable', 'captivity', 'carefree', 'century', 'chamber', 'circular', 'coax', 'column', 'communicate', 'competition', 'complete',
    'concentrate', 'concern', 'conclude', 'confuse', 'congratulate', 'considerable', 'content', 'contribute', 'crafty', 'create', 'demonstrate', 'descend', 'desire', 'destructive', 'develop', 'disaster',
    'disclose', 'distract', 'distress', 'dusk', 'eager', 'ease', 'entertain', 'entire', 'entrance', 'envy', 'essential', 'extraordinary', 'flexible', 'focus', 'fragile', 'frantic', 'frequent', 'frontier',
    'furious', 'generosity', 'hail', 'hardship', 'heroic', 'host', 'humble', 'Impact', 'increase', 'indicate', 'inspire', 'instant', 'invisible', 'jagged', 'lack', 'limb', 'limp', 'manufacture', 'master',
    'mature', 'meadow', 'mistrust', 'mock', 'modest', 'noble', 'orchard', 'outstanding', 'peculiar', 'peer', 'permit', 'plead', 'plentiful', 'pointless', 'portion', 'practice', 'precious', 'prefer',
    'prepare', 'proceed', 'queasy', 'recent', 'recognize', 'reduce', 'release', 'represent', 'request', 'resist', 'response', 'reveal', 'routine', 'severe', 'shabby', 'shallow', 'sole', 'source', 'sturdy',
    'surface', 'survive', 'terror', 'threat', 'tidy', 'tour', 'tradition', 'tragic', 'typical', 'vacant', 'valiant', 'variety', 'vast', 'venture', 'weary'
]
letter_frequency_list = [letter for word in background_words_list for letter in word]

In [9]:

wb = Workbook()

# Grab the active worksheet
ws = wb.active

words_list = [
    'SCRIPTURE', 'CREATURE', 'STRUCTURE', 'SIGNATURE', 'TEMPERATURE', 'ASTHMA', 'OFTEN', 'EPISTLE', 'SOLEMN', 'ISLAND',
    'KANSAS', 'KENTUCKY', 'APPLAUSE', 'EMPHASIS', 'FUNCTION', 'HAPPINESS', 'MEMORIAL', 'MIRROR', 'REPUTATION', 'VOLUME'
]
size_fac = 3#g.get_puzzle_dim(words_list)
print(size_fac)
rows_list = g.gen_word_search(words_list, directions=EASY_DIRECTIONS, size_fac=size_fac)
print(nu.get_shape(rows_list))

# Assign individual letters directly to cells
for row, columns_list in enumerate(rows_list):
    for column, value in enumerate(columns_list):
        if (value == ' '): value = random.choice(letter_frequency_list).upper()
        d = ws.cell(row=row+1, column=column+1, value=value)

# Assign individual words directly to the right of the letter cells
row = 1
column += 3
for word in words_list:
    d = ws.cell(row=row, column=column, value=word)
    row += 2

# Save and open the file
xlsx_dir = '../saves/xlsx'
counter = 1
file_name = f'Word_Search_Puzzle{counter:02d}.xlsx'
file_path = osp.abspath(osp.join(xlsx_dir, file_name))
while osp.exists(file_path):
    counter += 1
    file_name = f'Word_Search_Puzzle{counter:02d}.xlsx'
    file_path = osp.abspath(osp.join(xlsx_dir, file_name))
wb.save(file_path)
!"C:\Program Files\Microsoft Office\root\Office16\EXCEL.EXE" "{file_path}"

3
(21, 21)



Make the widths of the single-letter columns 1.89 (24 pixels) to keep the search words on the same page when printing. To get the puzzle to take up the whole page width, you have to rearrange the search words so they are in two equal columns on the second page, and the widths of the single-letter columns need to be changed to 3.33 (37 pixels), and the row heights to 22.20 (37 pixels). Select the entire puzzle and make it Middle Align and Center. To get the puzzle to take up the whole page width, change the size of the Calibri font to 18.